
# 0. The evolution of plasma state prediction

In this notebook, we attempt to build a Long short-term memory (LSTM) model to predict time series.

Thomson scattering is an important tool for plasma diagnostics in nuclear fusion facilities. Evolution of the Thomson Scattering data mapping the electron temperature and density of the profiles are predicted using an LSTM model. The model takes in the first 8 time instances of the profile and then maps the next 4 time instances. ​



In [ ]:
# tensorflow
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import EarlyStopping

# check version
print('Using TensorFlow v%s' % tf.__version__)
acc_str = 'accuracy' if tf.__version__[:2] == '2.' else 'acc'


# helpers
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
from collections import Counter
from scipy import asarray as ar,exp
from sklearn.model_selection import train_test_split
from sklearn import mixture
from sklearn import cluster
from sklearn.decomposition import PCA
import pandas as pd
import os.path
import matplotlib.patches as mpatches
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import MeanShift
import os
from IPython.display import clear_output
from tqdm import tqdm
import h5py
from os.path import join

# need some certainty in data processing
np.random.seed(1234)
tf.random.set_seed(1234)

data_path = 'data'


## Google Cloud Storage Boilerplate

The following two cells have some boilerplate to mount the Google Cloud Storage bucket containing the data used for this notebook to your Google Colab file system. **Even you are not using Google Colab, please make sure you run these two cells.** 

To access the data from Google Colab, you need to:

1. Run the first cell;
2. Follow the link when prompted (you may be asked to log in with your Google account);
3. Copy the Google SDK token back into the prompt and press `Enter`;
4. Run the second cell and wait until the data folder appears.

If everything works correctly, a new folder called `sciml-workshop-data` should appear in the file browser on the left. Depending on the network speed, this may take one or two minutes. Ignore the warning "You do not appear to have access to project ...". If you are running the notebook locally or you have already connected to the bucket, these cells will have no side effects.

In [ ]:
# variables passed to bash; do not change
project_id = 'sciml-workshop'
bucket_name = 'sciml-workshop'
colab_data_path = '/content/sciml-workshop-data/'

try:
    from google.colab import auth
    auth.authenticate_user()
    google_colab_env = 'true'
    data_path = colab_data_path
except:
    google_colab_env = 'false'
    ###################################################
    ######## specify your local data path here ########
    ###################################################
    with open('../local_data_path.txt', 'r') as f: data_path = f.read().splitlines()[0]

In [ ]:
%%bash -s {google_colab_env} {colab_data_path} {project_id} {bucket_name}

# running locally
if ! $1; then
    echo "Running notebook locally."
    exit
fi

# already mounted
if [ -d $2 ]; then
    echo "Data already mounted."
    exit
fi

# mount the bucket
echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
apt -qq update
apt -qq install gcsfuse
gcloud config set project $3
mkdir $2
gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 $4 $2

# 1. Load the dataset

### Read in the Thomson scattering data

Use pandas to read the hdf 'Fusion/fusion_inputs.h5' and 'Fusion/fusion_targets.h5'. 'fusion_inputs.h5' includes six time series data: plasma current, toroidal mag field, z position, aspect ratio (major radius/minor radius), gas outboard and aux heat. 'fusion_targets.h5' includes electron density and electron temperature. All time series follow the same times and shot number 

**Suggested Answer** 

<details> <summary>Show / Hide</summary> 
<p>
    
```python
with h5py.File(join(data_path, 'Fusion/fusion_inputs.h5'), 'r') as F:

    current_data = {}
    for i in range(len(np.array(F['current_data']))):
        current_data[i] = np.array(F['current_data'][str(i)])
        
    mag_data = {}
    for i in range(len(np.array(F['mag_data']))):
        mag_data[i] = np.array(F['mag_data'][str(i)])
    
    Z_data = {}
    for i in range(len(np.array(F['Z_data']))):
        Z_data[i] = np.array(F['Z_data'][str(i)])
    
    aspect_data = {}
    for i in range(len(np.array(F['aspect_data']))):
        aspect_data[i] = np.array(F['aspect_data'][str(i)])

    gas_data = {}
    for i in range(len(np.array(F['gas_data']))):
        gas_data[i] = np.array(F['gas_data'][str(i)])

    heat_data = {}
    for i in range(len(np.array(F['heat_data']))):
        heat_data[i] = np.array(F['heat_data'][str(i)])
        
        

with h5py.File(join(data_path, 'Fusion/fusion_targets.h5'), 'r') as F:

    shot_num = {}
    for i in range(len(np.array(F['shot_num']))):
        shot_num[i] = int(np.array(F['shot_num'][str(i)]))
        
    time_data = {}
    for i in range(len(np.array(F['time_data']))):
        time_data[i] = np.array(F['time_data'][str(i)])
    
    density_data = {}
    for i in range(len(np.array(F['density_data']))):
        density_data[i] = np.array(F['density_data'][str(i)])
    
    temperature_data = {}
    for i in range(len(np.array(F['temperature_data']))):
        temperature_data[i] = np.array(F['temperature_data'][str(i)])
```
    
</p>
</details>

In [ ]:
e_temp = temperature_data
e_density = density_data

### combine signals into one stacked array for each

In [ ]:
current_stack = np.hstack(list(current_data.values()))
mag_stack = np.hstack(list(mag_data.values()))
Z_stack = np.hstack(list(Z_data.values()))
aspect_stack = np.hstack(list(aspect_data.values()))
gas_stack = np.hstack(list(gas_data.values()))
heat_stack = np.hstack(list(heat_data.values()))

### Normalise each signal array by their maximum value

**Suggested Answer** 

<details> <summary>Show / Hide</summary> 
<p>
    
```python
current_stack = current_stack/np.max(current_stack)
mag_stack = mag_stack/np.max(mag_stack)
Z_stack = Z_stack/np.max(Z_stack)
aspect_stack = aspect_stack/np.max(aspect_stack)
gas_stack = gas_stack/np.max(gas_stack)
heat_stack = heat_stack/np.max(heat_stack)
```
    
</p>
</details>

### Combine all signals

In [ ]:
all_signals = np.column_stack((current_stack,mag_stack,Z_stack,aspect_stack,gas_stack,heat_stack))
np.shape(all_signals)

### Create array of radial profiles (temperature and density) across all shots

In [ ]:
temp_flat = []
for i in range(len(e_temp)):
    for j in range(len(e_temp[i])):
        temp_flat.append(e_temp[i][j])
        
density_flat = []
for i in range(len(e_density)):
    for j in range(len(e_density[i])):
        density_flat.append(e_density[i][j])

### Reshape array of radial profiles into 2D temporal/radial arrays for each shot

In [ ]:
e_temp_reshaped = []
e_density_reshaped = []
b = []
a = 0
for i in range(len(e_temp)):
    b.append(a)
    b_sum = np.sum(b)
    a = len(e_temp[i])
    e_temp_reshaped.append(temp_flat[b_sum:a+b_sum])
    e_density_reshaped.append(density_flat[b_sum:a+b_sum])

### Combine density and temperature

In [ ]:
new_temp = temp_flat/np.max(temp_flat) # normalise
new_density = density_flat/np.max(density_flat)
new_temp = np.reshape(new_temp,(len(new_temp),110,1))
new_density = np.reshape(new_density,(len(new_density),110,1))
target_all_flat = np.concatenate([new_temp,new_density],axis=2)
print(np.shape(target_all_flat))
training_shots_all = target_all_flat.reshape(len(target_all_flat),220)
print(np.shape(training_shots_all))

In [ ]:
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

### Split a multivariate sequence into samples using split_sequences

We will have a gap for a furture event prediction. 

In [ ]:
n_steps = 8
X, x1 = split_sequences(all_signals,n_steps) # X = training data, all_signals or all_signals_flat_test
print(np.shape(X))

In [ ]:
## delete end rows of training array depending on gap size
gap = 4
X = X[:-gap]
print(np.shape(X))

In [ ]:
## make target e_temp array 
e_all_target = training_shots_all[n_steps+gap:] ## e_all_flat or training_shots_all
y = e_all_target.reshape(len(e_all_target),220) # use if training on all data
y = e_all_target ## use for surrogate dataset
print(np.shape(y))

# 2. Build the network

In this case, the inputs are time series of six features and outputs are electron density and electron temperature. We suggest to build a neural network using dense and lstm layers. In order to stack LSTM layers, return_sequences should be True.



**Suggested Answer** 

<details> <summary>Show / Hide</summary> 
<p>
    
```python
n_features = X.shape[2]

# define model
model = Sequential()
model.add(Dense(n_features, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(n_features, activation='relu'))
model.add(LSTM(12, activation='relu', return_sequences=True))
model.add(layers.Dropout(0.2))
model.add(LSTM(24, activation='relu', return_sequences=True))
model.add(layers.Dropout(0.2))
model.add(LSTM(48, activation='relu', return_sequences=True))
model.add(layers.Dropout(0.2))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(layers.Dropout(0.2))
model.add(LSTM(200, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(Dense(220))
model.compile(optimizer='adam', loss='mse')
print(model.summary())
```
    
</p>
</details>


### Train the model with EarlyStopping

**Suggested Answer** 

<details> <summary>Show / Hide</summary> 
<p>
    
```python
# fit model
stopper=EarlyStopping( monitor ="val_loss", min_delta=0.0,verbose=1,
                      mode="min",
                      restore_best_weights=True,patience=2)

history = model.fit(X, y, epochs=50, batch_size=256, shuffle=True, validation_split=1/12.,callbacks=[stopper])
```
    
</p>
</details>

---

# 3. Analyse results 

### Check training history

**Suggested Answer** 

<details> <summary>Show / Hide</summary> 
<p>
    
```python
# loss curves
plt.figure(figsize=(8,5))
plt.plot(history.history['loss'],linewidth=2.5)
plt.plot(history.history['val_loss'],linewidth=2.5)
plt.ylabel('Loss value',fontsize=14)
plt.xlabel('Epoch',fontsize=14)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(['Training loss', 'Validation loss'], loc='upper right',fontsize=14)
plt.show()
```
    
</p>
</details>

### Predict and compare errors

**Suggested Answer** 

<details> <summary>Show / Hide</summary> 
<p>
    
```python
y_pred = model.predict(X)
print(np.shape(y_pred))
```
    
</p>
</details>

In [ ]:
## flattened array for comparison

e_all_compare = e_all_target[n_steps+gap:]
e_all_compare = e_all_compare.reshape(len(e_all_compare),220)
np.shape(e_all_compare)

In [ ]:
## compare to original decoded representation

errors = []
errors_t = []
errors_d = []

for i in range(0,len(e_all_compare)):
    subtract = (e_all_target[i] - y_pred[i])**2
    subtract_reshaped = subtract.reshape(110,2)
    
    mse = np.mean(subtract)
    subtract_mse_t = np.mean(subtract_reshaped[:,0])
    subtract_mse_d = np.mean(subtract_reshaped[:,1]) ## mse for density and temp
    
    errors.append(mse)
    errors_t.append(subtract_mse_t)
    errors_d.append(subtract_mse_d)
    
plt.figure(figsize=(15,10))
plt.plot(errors)

### Display the mean of errors of temeprature and density

**Suggested Answer** 

<details> <summary>Show / Hide</summary> 
<p>
    
```python
print(np.mean(errors_t)*100,'%') ##temp
print(np.mean(errors_d)*100,'%') ##density
```
    
</p>
</details>

### Reshape array of predicted radial profiles into 2D temporal/radial arrays for each shot

In [ ]:
y_pred = y_pred.reshape(len(y_pred),110,2)

temp_pred_reshaped = []
density_pred_reshaped = []
b = []
b1 = len(e_temp[0])-(n_steps+gap)
temp_pred_reshaped.append(y_pred[0:b1,:,0]*np.max(temp_flat)) ## *np.max(e_temp_flat) to un-normalise
density_pred_reshaped.append(y_pred[0:b1,:,1]*np.max(density_flat))
b.append(b1)
a = 0
for i in tqdm(range(1,len(e_temp))):
    b.append(a)
    b_sum = np.sum(b)
    a = len(e_temp[i])
    temp_pred_reshaped.append(y_pred[b_sum:a+b_sum,:,0]*np.max(temp_flat)) 
    density_pred_reshaped.append(y_pred[b_sum:a+b_sum,:,1]*np.max(density_flat)) 

In [ ]:
## compare predicted to original radial profiles (temperature)

# time vs. radius
from random import randrange
shot_index = randrange(len(shot_num))
print('shot number = ',shot_num[shot_index])

fig = plt.figure(figsize=(12,8))
fig.add_subplot(1,2,1)
plt.imshow(e_temp_reshaped[shot_index],vmin=0,vmax=np.max(temp_pred_reshaped[shot_index]),origin='lower')
plt.title('Shot '+str(shot_num[shot_index]))
plt.xlabel('Radial Measurement')
plt.ylabel('Time Measurement')
plt.colorbar(label='Temperature [eV]')

fig.add_subplot(1,2,2)
plt.subplots_adjust(wspace=0.25)
plt.imshow(temp_pred_reshaped[shot_index],vmin=0,vmax=np.max(temp_pred_reshaped[shot_index]),origin='lower')
plt.title('Predicted Shot '+str(shot_num[shot_index]))
plt.xlabel('Radial Measurement')
plt.ylabel('Time Measurement')
plt.colorbar(label='Temperature [eV]')

## compare predicted to original radial profiles (density)

# time vs. radius

fig = plt.figure(figsize=(12,8))
fig.add_subplot(1,2,1)
plt.imshow(e_density_reshaped[shot_index],vmin=0,vmax=np.max(density_pred_reshaped[shot_index]),origin='lower')
plt.title('Shot '+str(shot_num[shot_index]))
plt.xlabel('Radial Measurement')
plt.ylabel('Time Measurement')
plt.colorbar(label='Density [m-3]')

fig.add_subplot(1,2,2)
plt.imshow(density_pred_reshaped[shot_index],vmin=0,vmax=np.max(density_pred_reshaped[shot_index]),origin='lower')
plt.title('Predicted Shot '+str(shot_num[shot_index]))
plt.xlabel('Radial Measurement')
plt.ylabel('Time Measurement')
plt.colorbar(label='Density [m-3]')

---

# 4. Exercises

### Let's try doing some hyper-parameter tuning.  

* Different step sizes for time series data
* Different gap sizes
